Magpylib makes use of vectorized computation (massive speedup). This requires that you hand over all field computation instances (multiple objects with multiple positions (=paths)) at the same time to getB or getH. Avoid Python loops at all costs !!!

In [1]:
import magpylib as magpy

# Create a Cuboid magnet with magnetic polarization
# of 1000 mT pointing in x-direction and sides of
# 1,2 and 3 mm respectively.

cube = magpy.magnet.Cuboid(magnetization=(1000,0,0), dimension=(1,2,3))

# Create a Sensor for measuring the field

sensor = magpy.Sensor()

In [2]:
# By default, the position of a Magpylib object is
# (0,0,0) and its orientation is the unit rotation,
# given by a scipy rotation object.

print(cube.position)                   # -> [0. 0. 0.]
print(cube.orientation.as_rotvec())    # -> [0. 0. 0.]

# Manipulate object position and orientation through
# the respective attributes:

from scipy.spatial.transform import Rotation as R
cube.position = (1,0,0)
cube.orientation = R.from_rotvec((0,0,45), degrees=True)

print(cube.position)                            # -> [1. 0.  0.]
print(cube.orientation.as_rotvec(degrees=True)) # -> [0. 0. 45.]

# Apply relative motion with the powerful `move`
# and `rotate` methods.
sensor.move((-1,0,0))
sensor.rotate_from_angax(angle=-45, axis='z')

print(sensor.position)                            # -> [-1.  0.  0.]
print(sensor.orientation.as_rotvec(degrees=True)) # -> [ 0.  0. -45.]

[0. 0. 0.]
[0. 0. 0.]
[1. 0. 0.]
[ 0.  0. 45.]
[-1.  0.  0.]
[  0.   0. -45.]


In [5]:
# Use the `show` function to view your system
# through Matplotlib, Plotly or Pyvista backends.

magpy.show(cube, sensor, backend='plotly')

In [4]:
# Compute the B-field in units of mT for some points.

points = [(0,0,-1), (0,0,0), (0,0,1)]
B = magpy.getB(cube, points)

print(B.round()) # -> [[263.  68.  81.]
                 #     [276.  52.   0.]
                 #     [263.  68. -81.]]

# Compute the H-field in units of kA/m at the sensor.

H = magpy.getH(cube, sensor)

print(H.round()) # -> [220.  41.   0.]

[[263.  68.  81.]
 [276.  52.   0.]
 [263.  68. -81.]]
[51. 24.  0.]


In [10]:
import numpy as np
import matplotlib.pyplot as plt
from magpylib.source.magnet import Box ,Cylinder
from magpylib import Collection , displaySystem

# create magnets
x1 = Box(mag =(0 ,0 ,600) , dim =(3 ,3 ,3), pos =(-4,0,3))
x2 = Cylinder (mag =(0 ,0 ,500) , dim =(3 ,5))

# create collection
c = Collection (x1 ,x2)

#manipulate magnets individually
x1.rotate (45 ,(0 ,1 ,0) , anchor =(0 ,0 ,0))
x2.move ((5,0,-4))

# manipulate collection
c.move ((-2,0,0))

# calculate B-field on a grid
xs = np.linspace ( -10 ,10 ,33)
zs = np.linspace ( -10 ,10 ,44)
POS = np.array ([(x,0,z) for z in zs for x in xs])
Bs = c.getB(POS).reshape (44 ,33 ,3) #<--
VECTORIZED

# create figure
fig = plt.figure(figsize =(9 ,5))
ax1 = fig. add_subplot (121 , projection ='3d') # 3Daxis
ax2 = fig. add_subplot (122) # 2Daxis

# display system geometry on ax1
displaySystem (c, subplotAx =ax1 , suppress=True)

# display field in xz-plane using matplotlib
X,Z = np.meshgrid (xs ,zs)
U,V = Bs[:,:,0], Bs[: ,: ,2]
ax2. streamplot (X, Z, U, V, color=np.log(U**2+V**2)
)

plt.show ()

ModuleNotFoundError: No module named 'magpylib.source'